# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-11-17 10:31:21.250608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 10:31:26.348554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, maxlen=150, dtype='float32')
X_test_pad = pad_sequences(X_test_token, maxlen=150, dtype='float32')


vocab_size = len(tokenizer.word_index)

In [3]:
vocab_size

30419

In [17]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'i': 9,
 'it': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'but': 17,
 'movie': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'his': 23,
 'are': 24,
 'have': 25,
 'one': 26,
 'be': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'they': 32,
 'an': 33,
 'so': 34,
 'like': 35,
 'who': 36,
 'from': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'if': 41,
 'out': 42,
 'about': 43,
 "it's": 44,
 'has': 45,
 'what': 46,
 'some': 47,
 'there': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'no': 53,
 'up': 54,
 'she': 55,
 'my': 56,
 'time': 57,
 'even': 58,
 'which': 59,
 'would': 60,
 'really': 61,
 'only': 62,
 'had': 63,
 'story': 64,
 'me': 65,
 'see': 66,
 'can': 67,
 'their': 68,
 'were': 69,
 'well': 70,
 'than': 71,
 'much': 72,
 'get': 73,
 'do': 74,
 'great': 75,
 'been': 76,
 'we': 77,
 'first': 78,
 'bad': 79,
 'because': 80,
 'into': 81,
 'other': 82,

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [23]:
from tensorflow.keras import Sequential, layers

def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size+1, output_dim=10, mask_zero=True, input_length=150))
    model.add(layers.Conv1D(16,3))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model_cnn = init_cnn_model(vocab_size)

❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [24]:
model_cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 150, 10)           304200    
                                                                 
 conv1d_4 (Conv1D)           (None, 148, 16)           496       
                                                                 
 flatten_4 (Flatten)         (None, 2368)              0         
                                                                 
 dense_8 (Dense)             (None, 5)                 11845     
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 316,547
Trainable params: 316,547
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 8ms/step - loss: 0.6921 - accuracy: 0.5234 - val_loss: 0.6875 - val_accuracy: 0.5520
Epoch 2/20
55/55 [==============================] - 0s 5ms/step - loss: 0.5949 - accuracy: 0.8486 - val_loss: 0.6641 - val_accuracy: 0.6040
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2782 - accuracy: 0.9651 - val_loss: 0.5652 - val_accuracy: 0.7027
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0504 - accuracy: 0.9983 - val_loss: 0.5637 - val_accuracy: 0.7173
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0135 - accuracy: 1.0000 - val_loss: 0.5536 - val_accuracy: 0.7320
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.5611 - val_accuracy: 0.7373
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.5686 - val_accuracy: 0.7387
Epoch 8/20
55/55 [==

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [30]:
import gensim.downloader as api
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# load a word2vec embedding
word2vec_transfer = api.load("glove-wiki-gigaword-50")

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
            
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, maxlen=150, padding='post', dtype='float32')
X_test_pad_2 = pad_sequences(X_test_embed_2, maxlen=150, padding='post', dtype='float32')
    
    

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [31]:
def init_cnn_model_2():
    model = Sequential()
    model.add(layers.Conv1D(16, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn_2 = init_cnn_model_2()



In [32]:
es_2 = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_2.fit(X_train_pad_2, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es_2]
         )


res = model_cnn_2.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 9ms/step - loss: 0.8537 - accuracy: 0.5463 - val_loss: 0.7053 - val_accuracy: 0.5880
Epoch 2/20
55/55 [==============================] - 0s 5ms/step - loss: 0.5686 - accuracy: 0.7023 - val_loss: 0.6516 - val_accuracy: 0.6533
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.4422 - accuracy: 0.7994 - val_loss: 0.6495 - val_accuracy: 0.6733
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.3598 - accuracy: 0.8531 - val_loss: 0.6621 - val_accuracy: 0.6747
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2898 - accuracy: 0.8966 - val_loss: 0.6883 - val_accuracy: 0.6733
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2412 - accuracy: 0.9206 - val_loss: 0.7410 - val_accuracy: 0.6720
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.1865 - accuracy: 0.9537 - val_loss: 0.7951 - val_accuracy: 0.6760
Epoch 8/20
55/55 [==

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)